## Análise de Dados do Spotify

Esta análise que utiliza a API do Spotify para coletar informações sobre artistas, gêneros e faixas musicais. O objetivo é explorar os dados do Spotify e obter insights sobre a popularidade dos artistas, os gêneros mais populares e as características das faixas musicais.

Spotipy Documentation
https://spotipy.readthedocs.io/en/latest/ 

In [1]:
#Baixando a biblioteca do spotify
!pip install -q spotify

In [66]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Obtém as credenciais das variáveis de ambiente
client_id = os.getenv('client_secret')
client_secret = os.getenv('client_secret')

# Configure a autenticação
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [67]:
#Genêro Musical
generos = ['R&B', 'Soul']
artistas = [] #Lista para armazenar os artistas

#Loop para pegar os artistas dos generos
for genero in generos:
    results = sp.search(q='genre:"{}"'.format(genero), type='artist', limit=50)
    artistas.extend(results['artists']['items'])

In [68]:
# Criando lista com os dados desses artistas
dados_artistas = []

In [69]:
#Pegando informações sobre os artistas

for artista in artistas:
    artist_info = sp.artist(artista['id'])
    nome_artista = artist_info['name']
    popularidade = artist_info['popularity']
    generos = artist_info['genres']
    dados_artista = {                                   #criando dicionário com as informações
        'Nome do Artista': nome_artista,
        'Popularidade': popularidade,
        'Gêneros': generos
    }
    dados_artistas.append(dados_artista)

In [70]:
#Criando dataframe
df_artistas = pd.DataFrame(dados_artistas)
df_artistas.head(10)


,Nome do Artista,Popularidade,Gêneros
0,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]"
1,SZA,89,"[pop, r&b, rap]"
2,Chris Brown,86,"[r&b, rap]"
3,Brent Faiyaz,82,"[r&b, rap]"
4,Beyoncé,87,"[pop, r&b]"
5,Khalid,84,"[pop, pop r&b]"
6,Miguel,79,[r&b]
7,Summer Walker,79,"[r&b, rap]"
8,Bryson Tiller,78,"[kentucky hip hop, r&b, rap]"
9,Jhené Aiko,76,[r&b]


In [71]:
# Acessando informações das principais faixas

top_tracks_data = []

for artista in artistas:
    artist_info = sp.artist(artista['id'])
    nome_artista = artist_info['name']
    popularidade = artist_info['popularity']
    generos = artist_info['genres']
    top_tracks = sp.artist_top_tracks(artista['id'])
    
    for track in top_tracks['tracks']:
        track_info = {
            'Nome do Artista': nome_artista,
            'Popularidade': popularidade,
            'Gêneros': generos,
            'Faixa': track['name'],
            'Álbum': track['album']['name'],
            'Tempo de Reprodução': track['duration_ms'],
            'Popularidade da Faixa': track['popularity']
            
        }
        top_tracks_data.append(track_info)

# Criar DataFrame com as informações das principais faixas
df_top_tracks = pd.DataFrame(top_tracks_data)



In [72]:
df_top_tracks.head()

,Nome do Artista,Popularidade,Gêneros,Faixa,Álbum,Tempo de Reprodução,Popularidade da Faixa
0,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Creepin' (with The Weeknd & 21 Savage),HEROES & VILLAINS,221520,96
1,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Die For You (with Ariana Grande) - Remix,Starboy (Deluxe),232857,88
2,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Die For You,Starboy,260253,92
3,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Starboy,Starboy,230453,94
4,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Blinding Lights,After Hours,200040,94


In [73]:
#Dando um merge no daframe artista e track

df_final = pd.merge(df_artistas, \
                    df_top_tracks[['Nome do Artista', 'Popularidade', 'Faixa','Tempo de Reprodução','Popularidade da Faixa']], \
                    on=['Nome do Artista', 'Popularidade'], \
                    how='left')


In [74]:
df_final.head()

,Nome do Artista,Popularidade,Gêneros,Faixa,Tempo de Reprodução,Popularidade da Faixa
0,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Creepin' (with The Weeknd & 21 Savage),221520,96
1,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Die For You (with Ariana Grande) - Remix,232857,88
2,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Die For You,260253,92
3,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Starboy,230453,94
4,The Weeknd,95,"[canadian contemporary r&b, canadian pop, pop]",Blinding Lights,200040,94
